In [ ]:
## mount 해제
dbutils.fs.unmount("/mnt/flightdata")

Azure Data Lake Stoage Gen2 + Azure Databricks mount 연결 설정
---
* ABFS를사용하여 ADLS Gen2또는 Blob Storage 탑재
이 드라이버를 사용하면 많은 애플리케이션과 프레임워크에서 Data Lake Storage Gen2를 명시적으로 참조하는 코드 없이 Azure BlobStorage의 데이터에 액세스할 수 있습니다.
    인증을 위해 Azure AD(Azure Active Directory) 애플리케이션 서비스 주체를 사용하여 Azure 스토리지에 대한 액세스 구성을 진행해야 합니다.  
    https://learn.microsoft.com/ko-kr/azure/databricks/dbfs/mounts#mount-azure-storage

필요한 매개변수 값을 미리 text파일에 저장하여 값을 미리 담아두세요.

In [ ]:
configs {"fs.azure.account.auth.type": "OAuth",
       "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
       "fs.azure.account.oauth2.client.id": "<application-id>", --
       "fs.azure.account.oauth2.client.secret": "<client-secret>",
       "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/<tenant-id>/oauth2/token",
       "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

dbutils.fs.mount(
source = "abfss://<container-name>@<storage-account-name>dfs.core.windows.net/",
mount_point = "/mnt/flightdata",
extra_configs = configs)

데이터 읽어오기

In [ ]:
# Use the previously established DBFS mount point to read the data.
# create a data frame to read data.

flightDF = spark.read.format('csv').options(header='true', inferschema='true').load("/mnt/flightdata/folder1/*.csv")
display(flightDF)
# read the airline csv file and write the output to parquet format for easy query.
flightDF.write.mode("append").parquet("/mnt/flightdata/parquet/flights")
print("Done")

In [ ]:
import os.path
import IPython
from pyspark.sql import SQLContext
display(dbutils.fs.ls("/mnt/flightdata"))

dbutils. put() 사용

In [ ]:
dbutils.fs.put("/mnt/flightdata/1.txt", "Hello, World!", True)
dbutils.fs.ls("/mnt/flightdata/parquet/flights")

In [ ]:
# Copy this into a Cmd cell in your notebook.
acDF = spark.read.format('csv').options(
    header='true', inferschema='true').load("/mnt/flightdata/folder1/On_Time.csv")
acDF.write.parquet('/mnt/flightdata/parquet/airlinecodes')

# read the existing parquet file for the flights database that was created earlier
flightDF = spark.read.format('parquet').options(
    header='true', inferschema='true').load("/mnt/flightdata/parquet/flights")

# print the schema of the dataframes
acDF.printSchema()
flightDF.printSchema()

# print the flight database size
print("Number of flights in the database: ", flightDF.count())

# show the first 20 rows (20 is the default)
# to show the first n rows, run: df.show(n)
acDF.show(100, False)
flightDF.show(20, False)

# Display to run visualizations
# preferably run this in a separate cmd cell
display(flightDF)

In [ ]:
display(flightDF)

createOrReplaceTempView 생성

In [ ]:
# Run each of these queries, preferably in a separate cmd cell for separate analysis
# create a temporary sql view for querying flight information
FlightTable = spark.read.parquet('/mnt/flightdata/parquet/flights')
FlightTable.createOrReplaceTempView('FlightTable')

# create a temporary sql view for querying airline code information
AirlineCodes = spark.read.parquet('/mnt/flightdata/parquet/airlinecodes')
AirlineCodes.createOrReplaceTempView('AirlineCodes')

# using spark sql, query the parquet file to return total flights in January and February 2016
out1 = spark.sql("SELECT * FROM FlightTable WHERE Month=1")
NumJan2016Flights = out1.count()
out2 = spark.sql("SELECT * FROM FlightTable WHERE Month=2")
NumFeb2016Flights = out2.count()
print("Jan 2016: ", NumJan2016Flights, " Feb 2016: ", NumFeb2016Flights)
Total = NumJan2016Flights+NumFeb2016Flights
print("Total flights combined: ", Total)

# List out all the airports in Texas
#out = spark.sql(
#    "SELECT distinct(OriginCityName) FROM FlightTable where OriginStateName = 'Texas'")
#print('Airports in Texas: ', out.show(100))

# find all airlines that fly from Texas
#out1 = spark.sql(
#    "SELECT distinct(Reporting_Airline) FROM FlightTable WHERE OriginStateName='Texas'")
#print('Airlines that fly to/from Texas: ', out1.show(100, False))

매직명령 사용하기

In [ ]:
%sql
SELECT * FROM FlightTable
